# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [7]:
# Import libraries
import numpy as np
import pandas as pd

In [8]:
# Importing the dataset
pokemon = pd.read_csv("C:/Users/rapha/Documents/Data/lab-dataframe-calculations/your-code/pokemon.csv")
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [9]:
# your code here
pokemon1 = pokemon.groupby(["HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed"])["Total"].sum()
#pokemon[Totl] = pokemon.agg([])
pokemon1

HP   Attack  Defense  Sp. Atk  Sp. Def  Speed
1    90      45       30       30       40       236
10   55      25       35       45       95       265
20   10      55       15       20       80       200
             230      10       230      5        505
     15      20       10       55       80       200
                                                ... 
165  75      80       40       45       65       470
170  90      45       90       45       60       500
190  33      58       33       58       33       405
250  5       5        35       105      50       450
255  10      10       75       135      55       540
Name: Total, Length: 778, dtype: int64

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [10]:
# your code here
pokemon2 = pd.get_dummies(pokemon[["Type 1","Type 2"]])
pokemon2

,Type 1_Bug,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,...,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
797,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
798,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [15]:
# your code here
pokemon3 = pokemon2.corr(method ='kendall').rank()
pokemon3

,Type 1_Bug,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,...,Type 2_Ghost,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water
Type 1_Bug,36.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,3.0,...,28.0,34.0,24.0,3.0,5.0,34.0,2.0,33.0,35.0,30.0
Type 1_Dark,12.0,36.0,12.0,14.0,15.0,15.0,12.0,16.0,13.0,11.0,...,34.0,9.0,10.0,34.0,14.0,11.0,31.0,12.0,31.0,11.0
Type 1_Dragon,10.0,12.0,36.0,12.0,12.0,14.0,11.0,13.0,11.5,9.0,...,9.5,7.5,35.0,35.0,12.0,9.5,34.0,10.5,8.5,9.0
Type 1_Electric,6.5,7.5,7.5,36.0,7.5,8.5,6.5,8.5,7.5,6.5,...,31.0,30.0,4.0,30.0,35.0,5.5,4.0,5.5,33.5,32.0
Type 1_Fairy,20.0,21.0,22.0,21.0,36.0,23.0,20.0,26.0,22.0,18.0,...,19.0,19.0,18.0,21.0,25.0,23.0,16.0,22.0,18.0,21.0
Type 1_Fighting,14.5,15.5,14.5,16.5,17.5,36.0,14.5,18.0,15.0,14.0,...,12.0,11.5,11.0,12.5,16.5,13.5,33.0,14.0,32.0,12.5
Type 1_Fire,5.0,6.0,6.0,6.0,6.0,6.0,36.0,7.0,6.0,5.0,...,4.0,3.0,29.0,5.0,34.0,4.0,27.0,31.0,25.0,31.0
Type 1_Flying,23.5,25.5,26.5,24.0,32.5,30.5,25.0,36.0,29.0,26.0,...,26.0,28.5,26.0,28.0,32.0,30.5,24.5,28.5,26.5,27.5
Type 1_Ghost,10.0,12.0,10.5,12.0,13.5,12.5,9.5,14.5,36.0,9.0,...,9.5,35.0,8.5,9.5,12.0,33.0,7.0,10.5,8.5,9.0
Type 1_Grass,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,36.0,...,3.0,2.0,6.0,33.0,4.0,35.0,22.0,3.0,29.0,3.0


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [6]:
# your code here
Type 2 #I was not able to show, by coding, why I think it is the right answer